In [1]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import datetime as dt
import pandas as pd
import numpy as np

# Path to store Excel File
path = r'/Users/prabhavkumar/Desktop/Finance/AssetSigmas.xlsx'

# Dates
start = dt.datetime(2005, 1, 1)
end = dt.datetime.now()

# Assets 
indices = {'type': 'IDX', 'tickers': ['^GSPC', '^DJI', '^IXIC',]}    
coms = {'type': 'CMD', 'tickers': ['CL=F', 'GC=F', 'SI=F',]}       
fi = {'type': 'FI', 'tickers': ['^TNX',]}             
ccy = {'type': 'FX', 'tickers': ['BTC-USD', 'EURUSD=X',]}      
vol = {'type': 'VOL', 'tickers': ['^VIX', 'VXX',]}      
assets = [indices, coms, fi, ccy, vol]

In [2]:
data = {'Ticker': [], 
        'Type': [],
        'SD': [],
        '(-2 SD, -1 SD, PX Last, 1 SD, 2 SD)': [], 
        'Last Move (%)': [], 
        '(-2 SD, -1 SD, Avg Down Move)': [], 
        '(Avg Up Move, 1 SD, 2 SD)': [], 
        'Down Move Freq (%)': [],
        'No Move Freq (%)': [],
        'Up Move Freq (%)': [],
        'Min Move (%)': [],
        'Avg Abs Move (%)': [],
        'Max Move (%)': [],
       }

In [3]:
for asset_class in assets:
    for asset in asset_class['tickers']:
        # Gathering Asset Prices
        df = pdr.get_data_yahoo(asset,  start, end)["Adj Close"]
        
        # Calculating Returns
        returns = ((df/df.shift()) - 1) * 100
        returns.dropna(inplace=True)
        
        down_moves = np.array(list(filter(lambda x: x < 0, returns)))
        up_moves = np.array(list(filter(lambda x: x > 0, returns)))
        no_moves = np.array(list(filter(lambda x: x == 0, returns)))
        
        # Calculating Stats
        last_price = df.values[-1]
        mean = df.mean()
        std = df.std()
        
        avg_down_move = down_moves.mean()
        avg_down_move_std = down_moves.std()
        avg_up_move = np.array(up_moves).mean()
        avg_up_move_std = up_moves.std()

        # Specifying accuracy of data
        accuracy = 3
        
        # Updating Excel Data
        data['Ticker'].append(asset)
        
        data['Type'].append(asset_class['type'])
        
        data['SD'].append(round(std, accuracy-1))
        
        data['(-2 SD, -1 SD, PX Last, 1 SD, 2 SD)'].append(tuple(map(lambda x: round(x, accuracy-1), 
                                                                   [last_price - 2 * std, 
                                                                    last_price - 1 * std,
                                                                    last_price,
                                                                    last_price + 1 * std,
                                                                    last_price + 2 * std,])))          
        
        data['Last Move (%)'].append(round(returns.values[-1], accuracy))
        
        data['(-2 SD, -1 SD, Avg Down Move)'].append(tuple(map(lambda x: round(x, accuracy), 
                                                                   [avg_down_move - 2 * avg_down_move_std, 
                                                                    avg_down_move - 1 * avg_down_move_std,
                                                                    avg_down_move,]))) 
        
        data['(Avg Up Move, 1 SD, 2 SD)'].append(tuple(map(lambda x: round(x, accuracy), 
                                                                   [avg_up_move,
                                                                    avg_up_move + 1 * avg_up_move_std,
                                                                    avg_up_move + 2 * avg_up_move_std,])))
    
        data['Down Move Freq (%)'].append(round(100 * len(down_moves) / len(returns), accuracy))
        
        data['No Move Freq (%)'].append(round(100 * len(no_moves) / len(returns), accuracy))
        
        data['Up Move Freq (%)'].append(round(100 * len(up_moves) / len(returns), accuracy))
        
        data['Min Move (%)'].append(round(min(returns), accuracy))
        
        data['Avg Abs Move (%)'].append(round(abs(returns).mean(), accuracy))
        
        data['Max Move (%)'].append(round(max(returns), accuracy)) 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
excel_df = pd.DataFrame(data)
excel_df

,Ticker,Type,SD,"(-2 SD, -1 SD, PX Last, 1 SD, 2 SD)",Last Move (%),"(-2 SD, -1 SD, Avg Down Move)","(Avg Up Move, 1 SD, 2 SD)",Down Move Freq (%),No Move Freq (%),Up Move Freq (%),Min Move (%),Avg Abs Move (%),Max Move (%)
0,^GSPC,IDX,789.54,"(2679.41, 3468.95, 4258.49, 5048.03, 5837.57)",-1.587,"(-2.894, -1.855, -0.816)","(0.739, 1.635, 2.53)",45.077,0.048,54.875,-11.984,0.774,11.580
1,^DJI,IDX,6514.61,"(20932.82, 27447.43, 33962.04, 40476.65, 46991...",-2.092,"(-2.741, -1.757, -0.773)","(0.717, 1.588, 2.458)",45.773,0.072,54.155,-12.927,0.742,11.365
2,^IXIC,IDX,2945.92,"(8383.15, 11329.06, 14274.98, 17220.9, 20166.81)",-1.055,"(-3.107, -2.029, -0.951)","(0.858, 1.791, 2.723)",44.428,0.024,55.548,-12.321,0.899,11.806
3,CL=F,CMD,22.45,"(21.63, 44.07, 66.52, 88.97, 111.41)",0.151,"(-17.247, -9.614, -1.982)","(1.739, 3.933, 6.127)",47.761,0.315,51.924,-305.966,1.849,37.662
4,GC=F,CMD,383.95,"(1048.9, 1432.85, 1816.8, 2200.75, 2584.7)",0.448,"(-2.537, -1.674, -0.811)","(0.808, 1.596, 2.385)",47.067,0.533,52.399,-9.354,0.805,9.028
5,SI=F,CMD,7.26,"(10.68, 17.94, 25.2, 32.46, 39.72)",0.295,"(-4.835, -3.151, -1.467)","(1.405, 2.781, 4.156)",46.886,0.242,52.871,-17.754,1.431,12.971
6,^TNX,FI,1.11,"(-1.04, 0.07, 1.18, 2.29, 3.41)",-9.154,"(-5.517, -3.602, -1.688)","(1.82, 4.17, 6.521)",50.848,1.648,47.504,-29.320,1.723,49.900
7,BTC-USD,FX,12111.12,"(5457.98, 17569.1, 29680.22, 41791.34, 53902.46)",-3.691,"(-8.593, -5.568, -2.542)","(2.587, 5.524, 8.461)",45.630,0.040,54.330,-37.170,2.565,25.247
8,EURUSD=X,FX,0.12,"(0.93, 1.06, 1.18, 1.3, 1.42)",-0.229,"(-1.597, -1.026, -0.455)","(0.453, 1.078, 1.703)",49.707,0.468,49.824,-13.353,0.452,17.308
9,^VIX,VOL,9.33,"(3.85, 13.17, 22.5, 31.83, 41.15)",21.951,"(-12.66, -8.693, -4.725)","(6.223, 13.712, 21.201)",53.746,0.504,45.749,-29.573,5.387,115.598


In [5]:
excel_df.to_excel(path, index=False, header=True)